# ML visualization

In [2]:
# Import necessary libraries and apply settings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from IPython.display import Image
%matplotlib inline
import json
import fiona
from bokeh.io import show, output_file, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper
from bokeh.palettes import Reds6 as palette
from bokeh.plotting import figure, save
from bokeh.resources import CDN
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, shape
from shapely.prepared import prep
import pandas as pd
import geopandas as gpd
import json
import matplotlib as mpl
import pylab as plt

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool, LogColorMapper
from bokeh.plotting import figure
from bokeh.palettes import brewer

import panel as pn
import panel.widgets as pnw

incidents = pd.read_csv('fire_incidents2020.csv')#pd.read_csv('fire_incidents1year.csv')
incidents = incidents.append(pd.read_csv('fire_incidents2019.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2018.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2017.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2016.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2015.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2014.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2013.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2012.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2011.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2010.csv'));
incidents = incidents.append(pd.read_csv('fire_incidents2009.csv'));

/Users/kristin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/kristin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (52,83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/kristin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/kristin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,25,52,83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=intera

In [5]:
df = incidents

In [6]:
shapefile = 'Shapefiles/statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp'
# Read shapefile using Geopandas
gdf = gpd.read_file(shapefile)[['NAME', 'GSS_CODE', 'geometry']]

In [7]:
df_groupped = df.groupby(['CalYear', 'IncGeo_BoroughCode', 'IncidentGroup']).mean().reset_index()
merged = gdf.merge(df_groupped, left_on='GSS_CODE', right_on='IncGeo_BoroughCode', how='left')
merged = merged[['NAME', 'GSS_CODE', 'geometry', 'CalYear', 'AttendanceTimeSeconds', 'IncidentGroup']]
merged['AttendanceTimeSeconds'] = round(merged['AttendanceTimeSeconds'])
merged.head()

,NAME,GSS_CODE,geometry,CalYear,AttendanceTimeSeconds,IncidentGroup
0,Kingston upon Thames,E09000021,"POLYGON ((516401.600 160201.800, 516407.300 16...",2009,359.0,False Alarm
1,Kingston upon Thames,E09000021,"POLYGON ((516401.600 160201.800, 516407.300 16...",2009,377.0,Fire
2,Kingston upon Thames,E09000021,"POLYGON ((516401.600 160201.800, 516407.300 16...",2009,409.0,Special Service
3,Kingston upon Thames,E09000021,"POLYGON ((516401.600 160201.800, 516407.300 16...",2010,362.0,False Alarm
4,Kingston upon Thames,E09000021,"POLYGON ((516401.600 160201.800, 516407.300 16...",2010,377.0,Fire


In [8]:
def get_geodatasource(gdf):    
    """Get getjsondatasource from geopandas object"""
    json_data = json.dumps(json.loads(gdf.to_json()))
    return GeoJSONDataSource(geojson = json_data)

In [14]:
def bokeh_plot_map(gdf, def_low=0, def_high=4000, column=None, title=''):
    output_file("bokeh_kbl_years.html")
    """Plot bokeh map from GeoJSONDataSource """

    geosource = get_geodatasource(gdf) # convert to geojson
    palette = brewer['OrRd'][8]
    palette = palette[::-1]
    
    #vals = gdf[column]
    
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, 
                                     low = def_low,#vals.min(), 
                                     high = def_high #vals.max()
                                    )
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
                         location=(0,0), orientation='horizontal')

    tools = "pan,wheel_zoom,reset,hover,save"
    p = figure(title = title, plot_height=450 , plot_width=600, toolbar_location=None, tools=tools,
               x_axis_location=None, y_axis_location=None)
    
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    #Add patch renderer to figure
    p.patches('xs','ys', 
              source=geosource, 
              fill_alpha=1, 
              line_width=0.3, 
              line_color='black',  
              fill_color={'field' :column , 'transform': color_mapper})
    #Specify figure layout.
    p.add_layout(color_bar, 'below')
    
    hover3 = p.select_one(HoverTool)
    hover3.point_policy = "follow_mouse"
    hover3.tooltips = [
                      ("Borough name", "@NAME"),
                      ("Borough code", "@GSS_CODE"),
                      ("Average response (in sec)", "@AttendanceTimeSeconds"),
                      ("Year", "@CalYear"),
                      #("(Long, Lat)", "($x, $y)")
                     ]

    
    return p

In [15]:
# Define lowest and highest so they stay the same for all graphs
lowest_value = merged['AttendanceTimeSeconds'].min()
highest_value = merged['AttendanceTimeSeconds'].max()

In [16]:
def map_dash():
    """Map dashboard"""
    
    from bokeh.models.widgets import DataTable
    map_pane = pn.pane.Bokeh(width=600)
    data_select = pnw.Select(name='Incident Group',options=list(df.IncidentGroup.unique()), value='Fire')
    year_slider = pnw.IntSlider(start=2009,end=2020,value=2009)
    def update_map(event):
        #gdf, key = get_dataset(name='test',year=year_slider.value)        
        gdf = merged[(merged.CalYear == year_slider.value) & (merged.IncidentGroup == data_select.value)]
        
        map_pane.object = bokeh_plot_map(gdf, lowest_value, highest_value, 'AttendanceTimeSeconds', )        
        return
    year_slider.param.watch(update_map,'value')
    year_slider.param.trigger('value')
    data_select.param.watch(update_map,'value')
    app = pn.Column(pn.Row(data_select, year_slider),map_pane)
    return app

In [17]:

pn.extension()
map_dash()

Column
    [0] Row
        [0] Select(name='Incident Group', options=['False Alarm', ...], value='Fire')
        [1] IntSlider(end=2020, start=2009, value=2009)
    [1] Bokeh(Figure, width=600)